# Analyzing time series

While the Raven server specializes in hydrological modeling, the Finch server specializes in climate indicators, some of which are useful for stream flow series analysis. 

Here we'll test those indicators on a simple test file with around ten years of daily streamflow generated by a Raven simulation.

In [ ]:
%matplotlib inline

import os

import xarray as xr
from birdy import WPSClient

from ravenpy.utilities.testdata import get_file

# Connect to the finch server that will compute the statistics
url = os.environ.get(
    "FINCH_WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/finch/wps"
)
wps = WPSClient(url)

fn = str(get_file("hydro_simulations/raven-gr4j-cemaneige-sim_hmets-0_Hydrographs.nc"))

## Base flow index

The base flow index is the minimum 7-day average flow divided by the mean flow. 

In [ ]:
help(wps.base_flow_index)

The base flow index needs as input arguments the link to a NetCDF file storing the stream flow time series, the name of the stream flow variable, and the frequency at which the index is computed (`YS`: yearly, `QS-DEC`: seasonally). 

In [ ]:
resp_bfi = wps.base_flow_index(fn, variable="q_sim")
out, log, meta = resp_bfi.get(asobj=True)

from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()

out.base_flow_index.plot()

To compute generic statistics of a time series, use the `stats` process.

In [ ]:
help(wps.stats)

In [ ]:
# Here we compute the annual summer (JJA) minimum
resp_stats = wps.stats(fn, variable="q_sim", op="min", season="JJA")
out, log, meta = resp_stats.get(asobj=True)
out.qsummermin.plot()

## Frequency analysis

The process `freq_analysis` is similar to the previous stat sin the it fits a series of annual maxima or minima to a statistical distribution, and returns the values corresponding to different return periods. 

In [ ]:
help(wps.freq_analysis)

For example, computing the Q(2,7), the minimum 7-days streamflow of reccurrence two years, can be done using the following. 

In [ ]:
resp_fa = wps.freq_analysis(
    fn, variable="q_sim", mode="min", t=2, dist="gumbel_r", window=7
)

In [ ]:
out, log, meta = resp_fa.get(asobj=True)
out.q7minannual

An array of return periods can be passed. 

In [ ]:
resp_fa2 = wps.freq_analysis(
    fn, variable="q_sim", mode="max", t=(2, 5, 10, 25, 50, 100), dist="gumbel_r"
)
out, log, meta = resp_fa2.get(asobj=True)
out.q1maxannual.plot()

## Getting the parameters of the distribution and comparing the fit

It's sometimes more useful to store the fitted parameters of the distribution rather than storing only the quantiles. In the example below, we're first computing the annual maxima of the simulated time series, then fitting them to a gumbel distribution using the `fit` process. 

In [ ]:
ds = xr.open_dataset(fn)
ds

In [ ]:
import json

resp_stats2 = wps.stats(
    fn,
    variable="q_sim",
    op="max",
    check_missing="pct",
    missing_options=json.dumps({"pct": {"tolerance": 0.05}}),
)
ts = resp_stats2.get()[0]
print(ts)

In [ ]:
da = resp_stats2.get(asobj=True)[0]
da.qannualmax.isel(nbasins=0)

In [ ]:
resp = wps.fit(ts, dist="gumbel_r", check_missing="skip")
pa_fn = resp.get()[0]
pa = resp.get(True)[0]
pa.params

To see how the distribution defined by those parameters compare with the original data, Raven's `graph_fit` process can help by creating a graphic showing an histogram of the data, overlayed with the fitted probability density function. The left y-axis displays the density (pdf) while the right y-axis displays the histogram count.  

In [ ]:
url = os.environ.get(
    "WPS_URL", "https://pavics.ouranos.ca/twitcher/ows/proxy/raven/wps"
)
raven = WPSClient(url)

resp = raven.graph_fit(ts=ts, params=pa_fn)
resp.get(asobj=True)[0]